In [2]:
import sys
sys.path.append('..')

from llm_class import LLM
import pandas as pd
import requests

In [3]:
llm = LLM(model_id='bielik', key_path='../key.txt')

In [4]:
df = pd.read_csv("../data/dataset.csv", delimiter="|")
df

,en_text,pl_text
0,"Hey, everyone! Wanna hear about Fyresdal?","Ej, ludzie! Chcecie posłuchać o Fyresdal?"
1,"Hey, good folk! Wanna hear what our brave guar...","Ej, ludzie! Chcecie usłyszeć, co nasz dzielny ..."
2,"Shut your trap, dammit!","Stul pysk, do cholery!"
3,Four against one - and you came out alive.,Czterech na jednego. A dałeś radę.
4,Shame to let such a lad waste away behind bars...,"Szkoda, żeby taki chłop marniał za kratkami......"
...,...,...
8375,Farewell.,Bywaj.
8376,Farewell.,Bywaj.
8377,What do you have on offer?,Pokaż mi swoje towary.
8378,Show me what you have in stock.,Pokaż mi swoje towary.


In [5]:
prompts = [
    "'{english_text}' Przetłumacz na polski: ",
    "Przetłumacz na polski: '{english_text}'",
    "Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{english_text}'",
    "'{english_text}' Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie:",
    "Przetłumacz '{english_text}' na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie:",

    "Zdanie do przetłumaczenia: '{english_text}' \n" +
    "Przetłumacz podane zdanie na polski:",

    "Zdanie do przetłumaczenia: '{english_text}' \n" +
    "Przetłumacz podane zdanie na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie:",
]

In [ ]:
sampled_df = df.sample(n=1000, random_state=50, ignore_index=True)

for prompt_index, prompt_template in enumerate(prompts):

    print(prompt_index, prompt_template)

    for index, row in sampled_df.iterrows():

        english_text = row["en_text"]
        prompt = prompt_template.format(english_text=english_text)

        translation = llm.prompt_chat_custom_temperature( 
            prompt, 
            0.0
        )
        sampled_df.at[index, f'bielik_prompt{prompt_index}_translation'] = translation
        
        if (index + 1) % 100 == 0:
            print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/bielik_translations_prompt_templates.csv", sep='|', index=False)

0 '{english_text}' Przetłumacz na polski: 
Processed 100 out of 1000 samples...
Processed 200 out of 1000 samples...
Processed 300 out of 1000 samples...
Processed 400 out of 1000 samples...
Processed 500 out of 1000 samples...
Processed 600 out of 1000 samples...
Processed 700 out of 1000 samples...
Processed 800 out of 1000 samples...
Processed 900 out of 1000 samples...
Processed 1000 out of 1000 samples...
1 Przetłumacz na polski: '{english_text}'
Processed 100 out of 1000 samples...
Processed 200 out of 1000 samples...
Processed 300 out of 1000 samples...
Processed 400 out of 1000 samples...
Processed 500 out of 1000 samples...
Processed 600 out of 1000 samples...
Processed 700 out of 1000 samples...
Processed 800 out of 1000 samples...
Processed 900 out of 1000 samples...
Processed 1000 out of 1000 samples...
2 Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{english_text}'
Processed 100 out of 1000 samples...
Processed 200 out o

Few-shot

In [ ]:
examples =""" 
Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: 'I love programming in Python.'
Kocham programować w Pythonie.

Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: 'Shut your mouth you idiot.'
Zamknij się idioto.

Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: 'You scratch my back, I scratch yours.'
Ja ci pomogę, ty mi pomożesz.

Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: 'I hope you slept well. Tomorrow will be a long day.'
Mam nadzieję, że dobrze spałeś. Jutro będzie długi dzień.

Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: 'Die you ugly bastard.'
Giń ty brzydki skurwielu.
"""

In [10]:
few_shot_prompt = """
Wzroując się na przykładach przetłumacz zdanie na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie.

Przykłady:
{examples}

Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{english_text}'
"""

In [11]:
sampled_df = df.sample(n=1000, random_state=50, ignore_index=True)

for index, row in sampled_df.iterrows():

    english_text = row["en_text"]
    prompt = few_shot_prompt.format(english_text=english_text, examples=examples)

    translation = llm.prompt_chat_custom_temperature(prompt, 0.0)
    sampled_df.at[index, f'bielik_prompt7_translation'] = translation
    
    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/bielik_translations_prompt_few_shot.csv", sep='|', index=False)

Processed 100 out of 1000 samples...
Processed 200 out of 1000 samples...
Processed 300 out of 1000 samples...
Processed 400 out of 1000 samples...
Processed 500 out of 1000 samples...
Processed 600 out of 1000 samples...
Processed 700 out of 1000 samples...
Processed 800 out of 1000 samples...
Processed 900 out of 1000 samples...
Processed 1000 out of 1000 samples...


In [12]:
examples2 =""" 
Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: 'I love programming in Python.'
Tłumaczenie: Kocham programować w Pythonie.

Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: 'Shut your mouth you idiot.'
Tłumaczenie: Zamknij się idioto.

Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: 'You scratch my back, I scratch yours.'
Tłumaczenie: Ja ci pomogę, ty mi pomożesz.

Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: 'I hope you slept well. Tomorrow will be a long day.'
Tłumaczenie: Mam nadzieję, że dobrze spałeś. Jutro będzie długi dzień.

Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: 'Die you ugly bastard.'
Tłumaczenie: Giń ty brzydki skurwielu.
"""

In [13]:
few_shot_prompt2 = """
Wzroując się na przykładach przetłumacz zdanie na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie.

Przykłady:
{examples}

Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{english_text}'
Tłumaczenie:
"""

In [14]:
sampled_df = df.sample(n=1000, random_state=50, ignore_index=True)

for index, row in sampled_df.iterrows():

    english_text = row["en_text"]
    prompt = few_shot_prompt2.format(english_text=english_text, examples=examples)

    translation = llm.prompt_chat_custom_temperature(prompt, 0.0)
    sampled_df.at[index, f'bielik_prompt7_translation'] = translation
    
    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/bielik_translations_prompt_few_shot.csv", sep='|', index=False)

Processed 100 out of 1000 samples...
Processed 200 out of 1000 samples...
Processed 300 out of 1000 samples...
Processed 400 out of 1000 samples...
Processed 500 out of 1000 samples...
Processed 600 out of 1000 samples...
Processed 700 out of 1000 samples...
Processed 800 out of 1000 samples...
Processed 900 out of 1000 samples...
Processed 1000 out of 1000 samples...
